In [1]:
from __future__ import print_function
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import applications
from keras.layers import GlobalAveragePooling2D

D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ANACONDA\envs\keras-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarnin

In [2]:
import collections
import os
import cv2
import pandas as pd
import numpy as np
import random

In [41]:
NOT_ROTATE = -1
NUMBER_CLASSES = 4

def rot_corrections(label_name):
    if label_name == 'rotated_left':
        return cv2.ROTATE_90_CLOCKWISE
    elif label_name == 'upright':
        return NOT_ROTATE
    elif label_name == 'rotated_right':
        return cv2.ROTATE_90_COUNTERCLOCKWISE
    elif label_name == 'upside_down':    
        return cv2.ROTATE_180

def correct_image(image, rot_correction):
    if rot_correction == NOT_ROTATE:
        return image
    else:
        return cv2.rotate(image, rot_correction)    
    
def load_data(file_list):
    DATA_FOLDER = 'train'
    data = []
    for file in file_list:
        file_path = os.path.join(DATA_FOLDER, file)
        image = cv2.imread(file_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # to gray
        #image = image.reshape(image.shape[0], image.shape[1], 1)
        #image = cv2.resize(image, (256, 256), interpolation = cv2.INTER_AREA)
        #image = image.astype('float32')
        #image /= 255
        data.append(image)

    print('data_loaded')
    data = np.array(data)
    print('data shape: ', data.shape)
    return data
    
def preprocess(X, Y):
    """X: list of images
       Y: simple list of numbers"""
    #X =X.astype('float32')
    #X /= 255
    Y = np.array(Y)
    Y = Y.reshape(len(Y), 1)
    Y = keras.utils.to_categorical(Y, NUMBER_CLASSES)
    return X, Y

def labels_toonehot(label):
    if label == 'rotated_left':
        return 0
    elif label == 'rotated_right':
        return 1
    elif label == 'upright':
        return 2
    elif label == 'upside_down':
        return 3
#[1., 0., 0., 0.] - 'rotated_left'
#[0., 1., 0., 0.] - 'rotated_right'
#[0., 0., 1., 0.] - 'upright'
#[0., 0., 0., 1.] - 'upside_down'
#['rotated_left', 'rotated_right', 'upright', 'upside_down']    


# LOAD DATA

In [4]:
labels = pd.read_csv('train.truth.csv', sep=',')
labels['rot_corrections'] = labels.label.apply(rot_corrections)
data_y = labels.rot_corrections.tolist()

In [5]:
data_x = load_data(labels.fn.tolist())

data_loaded
data shape:  (48896, 64, 64, 3)


In [ ]:
x_files = labels.fn.tolist()[0:1000]
data_x = load_data(x_files)
x_train = data_x[800:]
x_test = data_x[:800]
data_y = data_y[0:1000]
data_x, data_y = preprocess(data_x, data_y)
y_train = data_y[800:]
y_test = data_y[:800]

# TRAIN TEST SPLIT

In [6]:
x_train = data_x[:34227]
x_test = data_x[34227:]
y_train = data_y[:34227]
y_test = data_y[34227:]

In [7]:
x_train, y_train = preprocess(x_train, y_train)
x_test, y_test = preprocess(x_test, y_test)

# CREATE AUGMENTED TRAIN FLOW

In [8]:
data_labels = labels.label.tolist()

In [ ]:
# FLOW

file_dir = os.path.join(os.getcwd(), 'flow')
k = 0
for image, label in zip(data_x, data_labels)
    image = cv2.resize(image, (256, 256), interpolation = cv2.INTER_AREA)
    file = os.path.join(file_dir, label)
    file = os.path.join(file, str(k) + '.jpg')
    cv2.imwrite(file, image)
    k += 1

In [9]:
# FLOW TRAIN
file_dir = os.path.join(os.getcwd(), 'flow_train')
k = 0
for image, label in zip(x_train, data_labels[:34227]):
    image = cv2.resize(image, (256, 256), interpolation = cv2.INTER_AREA)
    file = os.path.join(file_dir, label)
    file = os.path.join(file, str(k) + '.jpg')
    cv2.imwrite(file, image)
    k += 1

In [44]:
train_labels = [labels_toonehot(x) for x in data_labels[:34227]]

In [45]:
test_labels = [labels_toonehot(x) for x in data_labels[34227:]]

# MODEL DEFINITION

In [10]:
batch_size = 32
epochs = 1
data_augmentation = True
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'rotvision_trained_model.h5'
flow_folder = r"./flow_train/"

# CIFAR

In [ ]:
batch_size = 32
epochs = 100
data_augmentation = True
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'rotvision_trained_model.h5'

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUMBER_CLASSES))
model.add(Activation('softmax'))    

# RESNET

In [8]:
train_datagen = ImageDataGenerator()

In [19]:
train_generator = train_datagen.flow_from_directory(
    directory=r"./flow/",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)

Found 8 images belonging to 2 classes.


In [11]:
x, y = train_generator.next()

In [24]:
image = cv2.cvtColor(x[3], cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (256, 256), interpolation = cv2.INTER_AREA)

In [25]:
cv2.imwrite('resized-image.jpg', image)

True

In [11]:
base_model = applications.resnet50.ResNet50(
    weights= "imagenet", 
    include_top=False, 
    input_shape=(256, 256, 3))

In [12]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(NUMBER_CLASSES, activation= 'softmax')(x)
model = keras.Model(inputs = base_model.input, outputs = predictions)

In [13]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)  # 2.2.0 to 2.3.1  --> lr changed to learning_rate
                                                                            #I am using the old version here
    
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# RUN MODEL

In [14]:

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
########################################################################################        
        rescale=1./255,
########################################################################################        
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    #datagen.fit(x_train)
    datagen = ImageDataGenerator(rescale=1./255)
    train_generator = datagen.flow_from_directory(
        directory=flow_folder,
        target_size=(256, 256),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42)    
    
    
#    # Fit the model on the batches generated by datagen.flow().
#    model.fit_generator(datagen.flow(x_train, y_train,
#                                     batch_size=batch_size),
#                        epochs=epochs,
#                        validation_data=(x_test, y_test),
#                        workers=4)

    model.fit_generator(train_generator,
                        epochs=epochs,
                        workers=4)
#                        validation_data=train_generator,



Using real-time data augmentation.
Found 34227 images belonging to 4 classes.
Epoch 1/1
1070/1070 [==============================] - 560s 523ms/step - loss: 0.1380 - acc: 0.9528


In [ ]:
# Checagem do treino

In [48]:
resized_x = []
for i in range(100):
    res_x = cv2.resize(x_train[i], (256, 256), interpolation = cv2.INTER_AREA)
    res_x = np.array(res_x) / 255
    resized_x.append(res_x)
resized_x = np.array(resized_x)

In [49]:
ypred_train = model.predict(resized_x)

In [51]:
pred1 = np.argmax(ypred_train,axis=1)
truth1 = train_labels[0:100]
#truth1 = np.argmax(truth1,axis=1)

In [52]:
acertos = []
for pred, truth in zip(pred1, truth1):
    if pred == truth:
        acertos.append(1)
    else:
        acertos.append(0)

In [53]:
np.mean(acertos)

0.98

array([2, 3, 3, ..., 0, 0, 2], dtype=int64)

In [16]:
ypred = model.predict(resized_x)

In [54]:
resized_x = []
for i in range(100):
    res_x = cv2.resize(x_test[i], (256, 256), interpolation = cv2.INTER_AREA)
    res_x = np.array(res_x) / 255
    resized_x.append(res_x)
resized_x = np.array(resized_x)

In [55]:
pred1 = np.argmax(ypred,axis=1)
truth1 = test_labels[0:100]
#truth1 = np.argmax(truth1,axis=1)

In [56]:
acertos = []
for pred, truth in zip(pred1, truth1):
    if pred == truth:
        acertos.append(1)
    else:
        acertos.append(0)

In [57]:
np.mean(acertos)

0.91

In [22]:
x, y = train_generator.next()

In [33]:
x.shape

(32, 256, 256, 3)

In [47]:
i = 6
image = x[i] * 255
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2.imwrite(str(i) + '.jpg', image)
y[i]

array([0., 0., 1., 0.], dtype=float32)

In [ ]:
#[1., 0., 0., 0.] - 'rotated_left'
#[0., 1., 0., 0.] - 'rotated_right'
#[0., 0., 1., 0.] - 'upright'
#[0., 0., 0., 1.] - 'upside_down'
#['rotated_left', 'rotated_right', 'upright', 'upside_down']

In [43]:
collections.Counter(labels.label.tolist())

Counter({'rotated_left': 12266,
         'upright': 12212,
         'rotated_right': 12314,
         'upside_down': 12104})

In [52]:
a.sort()

In [53]:
a

['rotated_left', 'rotated_right', 'upright', 'upside_down']

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [58]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_json = model.to_json()
model_json_path = os.path.join(save_dir, model_name.split('.')[0] + '.json')
with open(model_json_path, 'w') as json_file:
    json_file.write(model_json)
    
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\Frederico\Desktop\rodando\saved_models\rotvision_trained_model.h5 


In [59]:
from keras.models import model_from_json
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'rotvision_trained_model.h5'

# load json and create model
model_json_path = os.path.join(save_dir, model_name.split('.')[0] + '.json')
model_path = os.path.join(save_dir, model_name)

json_file = open(model_json_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_path)

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)  # 2.2.0 to 2.3.1  --> lr changed to learning_rate
loaded_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])    

print("Loaded model from disk")

Loaded model from disk


In [60]:
resized_x = []
for i in range(100):
    res_x = cv2.resize(x_test[i], (256, 256), interpolation = cv2.INTER_AREA)
    res_x = np.array(res_x) / 255
    resized_x.append(res_x)
resized_x = np.array(resized_x)

In [62]:
ypred = loaded_model.predict(resized_x)

In [63]:
pred1 = np.argmax(ypred,axis=1)
truth1 = test_labels[0:100]
#truth1 = np.argmax(truth1,axis=1)

In [65]:
acertos = []
for pred, truth in zip(pred1, truth1):
    if pred == truth:
        acertos.append(1)
    else:
        acertos.append(0)
np.mean(acertos)

0.91